# Assignment 2 For Clustering: Sessa Empirical Estimator
- Read the Journals about the Sessa Empirical Estimator.

- Convert the R codes into Python Codes (use jupyter notebook).

- Using Simulated data or a real world datasets of your choice, perform the Sessa Empircal Estimator and generate some insights.
    - https://www.frontiersin.org/journals/pharmacology/articles/10.3389/fphar.2019.00383/full
    - https://archive.ics.uci.edu/

- The Sessa Empirical Estimator uses K-Means clustering (again recall the disadvantages of K-Means), try to substitute a different clustering algorithm, generate a new insight using the new clustering algorithm.

- Compare your results between Sessa Empirical Estimator using K-Means, and Sessa Empirical Estimator using the clustering algorithm of your choice.

- Deadline is this Sunday, Feb 23, 2022 at 11:59 pm

- Do this with your thesis partner.

- You can use any A.I. assistant.

#### Import equivalent libraries from R to Python

In [ ]:
library(AdhereR)
library(dplyr)
library(plyr)
library(lubridate)
library(latticeExtra)
library(data.table)
library(factoextra)
library(stats)

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from scipy.stats import norm
from sklearn import decomposition, preprocessing

#### Convert and generate simulated data from article provided
- https://github.com/Masswear/BeyondThresholds/blob/master/Code/functions.R

- Asked GPT for the equivalent R functions in Python
    - i.e what in the world qnorm() is

In [ ]:
def logistics(x, L=0, S=1, D=1, h=1, B=None):
    if B is None:
        B = x - D
    y = (h-(5*L))/(1 + np.exp(S*B)) + L
    return y

def med_events_sample(ntot, start_date="01.01.2022", tot_duration=2*365, disp_durations=[30, 60, 90], dist_durations=[0.3, 0.5, 0.2], dist=[0.1, 0.2, 0.2, 0.2, 0.2, 0.1]):
    def offset(group, n):
        if group == 1:
            L, U, m, s = -0.1, 0.2, 0.05, 0.1
            pL, pU = norm.cdf(L, m, s), norm.cdf(U, m, s)
            offset = norm.ppf(np.random.uniform(pL, pU, n), m, s)
        elif group == 2:
            L, U, m, s = -0.2, 1.2, 0, 1
            pL, pU = norm.cdf(L, m, s), norm.cdf(U, m, s)
            offset = norm.ppf(np.random.uniform(pL, pU, n), m, s)
        elif group == 3:
            L, U, m, s = 0.5, 1.5, 1, 1
            pL, pU = norm.cdf(L, m, s), norm.cdf(U, m, s)
            offset1 = norm.ppf(np.random.uniform(pL, pU, n), m, s)
            t = 2/n * np.arange(1, n+1)
            offset = t* offset1
        elif group == 4:
            L, U, m, s = 0.8, 1.2, 1, 0.1
            pL, pU = norm.cdf(L, m, s), norm.cdf(U, m, s)
            offset1 = norm.ppf(np.random.uniform(pL, pU, n), m, s)
            offset = logistics(x=np.arange(1, n+1), L=0.05, S=10, D=n, B=np.sin(2*np.arange(1, n+1)-n))*offset1
        elif group == 5:
            L, U, m, s = 0.5, 1.5, 1, 1
            pL, pU = norm.cdf(L, m, s), norm.cdf(U, m, s)
            offset1 = norm.ppf(np.random.uniform(pL, pU, n), m, s)
            offset = logistics(x=np.arange(1, n+1), L=0.05, S=-15, D=n/3)*offset1
        elif group == 6:
            n =  np.random.choice([2,3])
            L, U, m, s = -0.2, 0.8, 0.3, 1
            pL, pU = norm.cdf(L, m, s), norm.cdf(U, m, s)
            offset = norm.ppf(np.random.uniform(pL, pU, n), m, s)

        return offset

    def refills(x, group):
        initial_fill = 30
        offsets = offset(group=group)

        durations = np.random.choice(disp_durations, size=len(offsets)-1, replace=True, p=dist_durations)
        durations = np.insert(durations, 0, initial_fill)

        date = datetime.strptime(start_date, "%d.%m.%Y").date()

        refill_dates = [date] + [date + timedelta(days=int(sum(durations[:i+1]) + round(offsets[i] * durations[i], 0))) for i in range(len(durations))]

        df = pd.DataFrame({
            'GROUP': group,
            'PATIENT_ID': x, 
            'DATE': refill_dates[:-1],
            'DURATION': durations
        })

        return df

    ID_last = 0
    sample = pd.DataFrame()
    
    mean_duration = np.sum(np.array(disp_durations) * np.array(dist_durations))

    n = int(np.ceil((tot_duration/mean_duration) * 1.5))

    temp_samples = []

    for i in range(1, 6):
        num_pat = round(dist[i-1] * ntot)
        ID_first = ID_last + 1
        ID_last = ID_first + num_pat - 1
        group = pd.concat([refills(x, i, offset, disp_durations, dist_durations, start_date) for x in range(ID_first, ID_last + 1)])
        temp_samples.append(group)

    num_pat = ntot - ID_last
    ID_first = ID_last + 1
    ID_last = ID_first + num_pat - 1
    group = pd.concat([refills(x, 6, offset, disp_durations, dist_durations, start_date) for x in range(ID_first, ID_last + 1)])
    temp_samples.append(group)

    sample = pd.concat(temp_samples)
    print(sample)
        
            
        
            



